# Getting the Data Set

Before generating an HMM of the BPTI Kunitz domain is to acquire dataset of protein sequences containing the Kunitz domain. The dataset was acessed from the PDB. We **included** structures of the following properties:

1.   **Resolution**: we have chosen structures with a resolution **in the range of 0 - 3 Å**. Structures with higher resolution would **compromise the reliability** of our model. The treashold of **3 Å** is chosen since it can model the interaction bonds and the **distance between two carbons** in the backbone. Hydrogen bonds are crucial for the formation of secondary structures and protein-protein interactions. The **lenghts** of hydrogen bonds depend on the **polar residues** that participate in the interaction which is why we use such a  variable range.

2.   **PFAM domain**: we include only proteins that match with the PFAM ID PF00014, associated to Kunitz/Bovine pancreatic trypsin inhibitor (BPTI) domain.

3.   **Length of polymer chain**: we include only proteins with a length below 70 aa. Thus we avoid proteins that include the domain in their sequences. We include **merely the domains themselves**. For this step it is critical to research the literature to be able to determine the usual length of this kind of domain.

We could also add a filter to exclude mutated proteins - changed to increase stability for crystallization - or the study of pathological variances. In this analysis, we did however not filter for such special cases. Our set contained only few structures with engineered mutations which where in positions that should not compromise our research.

**QUERY**: Resolution (Å) = [ 0 - 3 ] AND ( Annotation Identifier = "PF00014" AND Annotation Type = "Pfam" ) AND Total Number of Polymer Residues (All Deposited Polymer Chains) < 100

Setting these options in the Advanced Search of the PDB webpage we end up with 39 protein structures. After that we must download the list of proteins setting as Tabular Report **"Sequence"**. The command below extracts the protein ID, the chain and the sequence and create a FASTA format from sequences in PDB. Moreover, we filtered all the structures composed by a number of residues between 50 and 70. After this filtering process there should be 25 structures in the FASTA file.

### Filterning for Sequences at Least 50 and Less than 70 Residues Long

In [9]:
sed 's/\, / /g' kunitz.csv | sed 's/\"//g' |awk -F ',' '{if($6>50 && $6<70) print ">"$1":"$2"\n"$5}' > kunitz_pdb.fasta

In [10]:
grep ">" kunitz_pdb.fasta | wc -l 

      25


In [2]:
cat kunitz_pdb.fasta

>3OFW:A
EAEASICSEPKKVGRCKGYFPRFYFDSETGKCTPFIYGGCGGNGNNFETLHQCRAICRLG
>4PTI:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>6Q61:A
KDRPSLCDLPADSGSGTKAEKRIYYNSARKQCLRFDYTGQGGNENNFRRTYDCARTCLYTA
>6PTI:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>1QLQ:A
RPDFCLEPPYAGACRARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCLRTCGGA
>1BTI:A
RPDFCLEPPYTGPCKARIIRYAYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>1G6X:A
RPDFCLEPPYAGACRARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCLRTCGGA
>5PTI:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>5YV7:A
WQPPWYCKEPVRIGSCKKQFSSFYFKWTAKKCLPFLFSGCGGNANRFQTIGECRKKCLGK
>1NAG:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRGNFKSAEDCMRTCGGA
>8PTI:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVGGGCRAKRNNFKSAEDCMRTCGGA
>7PTI:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLAQTFVYGGCRAKRNNFKSAEDAMRTCGGA
>9PTI:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>1DTX:A
QPRRKLCILHRNPGRCYDKIPAFYYNQKKKQCERFDWSGCGGNSNRFKTIEECRRTCIG
>1FAN:A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNAK

### Clustering the Sequences

This command allows us to cluster sequences according to the sequence identity (-S) and the lenght of the coverage (-L). Write why we choose 99 and 0.99. There are 14 clusters but in one of them contains proteins of the APP **not the BPTI domain** (see second line of PF00014pdb.clust &rarr; 3rd cell below second line)

In [1]:
grep ">" kunitz_pdb.fasta | wc -l

      25


In [4]:
~/blast-2.2.26/bin/blastclust -i kunitz_pdb.fasta -o PF0014pdb.clust -S 99 -L 0.99

May 11, 2020 11:05 PM Start clustering of 25 queries


In [23]:
# check out how the file looks like:
cat PF0014pdb.clust

1BPI:A 4PTI:A 5PTI:A 6PTI:A 9PTI:A 
2FJZ:A 2FK1:A 2FK2:A 2FMA:A 
1G6X:A 1K6U:A 1QLQ:A 
1KNT:A 1KTH:A 2KNT:A 
6Q61:A 
3OFW:A 
5YV7:A 
1DTX:A 
1BPT:A 
1BTI:A 
1FAN:A 
1NAG:A 
7PTI:A 
8PTI:A 


### Choosing the Most Representative or Best Structure of Each Cluster

Now we have to choose the **best structure of each cluster**. So we have to check wich of the structures has the lowest (=best) resolution. We downlowaded the csv that includes the [resolution](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22less_or_equal%22%2C%22negation%22%3Afalse%2C%22value%22%3A3%2C%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%7D%2C%22node_id%22%3A0%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_polymer_entity_annotation.annotation_id%22%2C%22negation%22%3Afalse%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22PF00014%22%7D%2C%22node_id%22%3A1%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_polymer_entity_annotation.type%22%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22Pfam%22%7D%2C%22node_id%22%3A2%7D%5D%2C%22label%22%3A%22nested-attribute%22%7D%2C%7B%22type%22%3A%22terminal%22%2C%22%22%3A%2275%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22less%22%2C%22negation%22%3Afalse%2C%22value%22%3A80%2C%22attribute%22%3A%22rcsb_entry_info.deposited_polymer_monomer_count%22%7D%2C%22node_id%22%3A3%7D%5D%7D%5D%2C%22label%22%3A%22text%22%7D%5D%2C%22label%22%3A%22query-builder%22%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A100%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22direction%22%3A%22asc%22%7D%5D%7D%2C%22request_info%22%3A%7B%22src%22%3A%22ui%22%2C%22query_id%22%3A%22ad4cf4dde4c5e69228827549b896acb6%22%7D%7D) and write a script to filter for the sequence with the best resolution of **each cluster**. They are few - so it could also be done manually...

### PDBe Fold Multiple Structure Alignment

Once we found the **best of each cluster** (only one per cluster), we save them in a single .txt file and go to [PDBe fold](https://www.ebi.ac.uk/msd-srv/ssm/). 
Check the 'multiple' box and then choose 'List of PDB codes'.
Upload the file and click submit query.

Then check out the matrices:

![](./imgs/1.png)

The higher the Q-score the better the similarity. The Q-score depends also on the size of the structure that we align - in our case it is a short domain. 

![](./imgs/2.png)

We examine the sequence id as well - there might be completely wrong seuquences in the set...

![](./imgs/3.png)


By scrutinizing the .fasta file of the MSA we are able to spot alignments which are **not meaningful**:


In [2]:
cat aln_all_clusters.fasta.seq


>PDB:5pti:A STRUCTURE OF BOVINE PANCREATIC TRYPSIN INHIBITOR. 
--rpdfcleppytgpckariiryfynakAGLCQTFV-YggcrakrnnfK---SAEDCMRTCGga------
-----------------------

>PDB:2fma:A STRUCTURE OF THE ALZHEIMER'S AMYLOID PRECURSOR PRO
------------------------eackFLHQERMD-Vc---------EthlHWHTVAKETCsekstnlh
dygmllpcgidkfrgvefvccpl

>PDB:1g6x:A ULTRA HIGH RESOLUTION STRUCTURE OF BOVINE PANCREAT
--rpdfcleppyagacrariiryfynakAGLCQTFV-YggcrakrnnfK---SAEDCLRTCGga------
-----------------------

>PDB:1kth:A THE ANISOTROPIC REFINEMENT OF KUNITZ TYPE DOMAIN C
--etdicklpkdegtcrdfilkwyydpnTKSCARFW-YggcggnenkfG---SQKECEKVCApv------
-----------------------

>PDB:6q61:A PORE-MODULATING TOXINS EXPLOIT INHERENT SLOW INACT
-drpslcdlpadsgsgtkaekriyynsaRKQCLRFD-YtgqggnennfR---RTYDCARTCLyt------
-----------------------

>PDB:3ofw:A CRYSTAL STRUCTURE OF RECOMBINANT KUNITZ TYPE SERIN
--easicsepkkvgrckgyfprfyfdseTGKCTPFI-YggcggngnnfE---TLHQCRAICR--------
-----------------------

>PDB:5yv7:A RACEMIC X-RAY STRUCTURE OF CALCICL

### One Sequence does NOT Contain BPTI Domain

After inspecting the ``` aln_all_clusters.fasta.seq``` file, we found one seq that contains the **APP** but **not** the BPTI. This sequence ('2FMA') is responsible for the bad alignment which is why it must be removed. 

### Removing Sequences that Don't Fit

We have to change the list of best clusters and remove 2FMA and re run the PDB fold with the updated list (```final_cluster_for_PDB_fold```). 

The new file contains a much better alignment :- see image below:

* Redo the clustering without the APP cluster --> there are no changes in the clusters.
* Show the HMM LOGO

![](./imgs/4.png)

redoing the clusterning with the BPTI only file:

In [20]:
~/blast-2.2.26/bin/blastclust -i kunitz_pdb.fasta -o PF0014pdb.clust -S 30

May 10, 2020  5:09 PM Start clustering of 25 queries


In [10]:
# check again how the new clusters look like:
cat seqfile.clust

tmpseq_1 tmpseq_10 tmpseq_11 tmpseq_16 tmpseq_17 tmpseq_18 tmpseq_2 tmpseq_3 tmpseq_4 tmpseq_5 tmpseq_7 
tmpseq_20 tmpseq_21 tmpseq_22 tmpseq_23 
tmpseq_12 tmpseq_15 tmpseq_8 
tmpseq_13 tmpseq_14 tmpseq_24 
tmpseq_9 
tmpseq_0 
tmpseq_19 
tmpseq_6 


### Removing it from the File Used for Clustering aswell:
first keeping one safety copy of the original ```file kunitz_pdb.fasta``` and only changeing that one

``` cp kunitz_pdb.fasta bpti_only_pdb.fasta```

``` head -42 bpti_only_pdb.fasta > only_bpti_pdb.fasta ``` this retaines only the sequences from line 1-42 the **4 APP seq are discared**.

Deleted ```bpti_only_pdb.fasta``` to avoid confusion with the new shorter file.

**``` final_cluster_for_PDB_fold ``` is the output of the second alignment with PDB_fold with only BPTI input**

### 'Cleaning' the Data

But first well turn all letters into capitals 

In [5]:
awk '{print toupper($1)}' newPDBfold_only_real_ones.fasta > bpti-kunitz-aln-clean.fasta 


In [3]:
cat bpti-kunitz-aln-clean.fasta
cp bpti-kunitz-aln-clean.fasta kunitz_pdb_hmm_input.fasta #making copy with name that indicates 
#that I'll later use it for hmmer  

grep '>' bpti-kunitz-aln-clean.fasta | wc -l # APP cluster was REMOVED it does not belong to the PF00014

# 2FMA, 2FJZ, 2FK1, 2FK2, 1FAN removed

>PDB:5PTI:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLCQTFVYGGCRA-KRNNFKSAEDCMRTCGGA
>PDB:1G6X:A
--RPDFCLEPP-YAGACRARIIRYFYNAKAGLCQTFVYGGCRA-KRNNFKSAEDCLRTCGGA
>PDB:1KTH:A
--ETDICKLPK-DEGTCRDFILKWYYDPNTKSCARFWYGGCGG-NENKFGSQKECEKVCAPV
>PDB:6Q61:A
-DRPSLCDLPA-DSGSGTKAEKRIYYNSARKQCLRFDYTGQGG-NENNFRRTYDCARTCLYT
>PDB:3OFW:A
--EASICSEPK-KVGRCKGYFPRFYFDSETGKCTPFIYGGCGG-NGNNFETLHQCRAICR--
>PDB:5YV7:A
WQPPWYCKEPV-RIGSCKKQFSSFYFKWTAKKCLPFLFSGCGG-NANRFQTIGECRKKCLGK
>PDB:1DTX:A
EPRRKLCILHR-NPGRCYDKIPAFYYNQKKKQCERFDWSGCGG-NSNRFKTIEECRRTCIG-
>PDB:1BPT:A
--RPDFCLEPP-YTGPCKARIIRYFANAKAGLCQTFVYGGCRA-KRNNFKSAEDCMRTCG--
>PDB:1BTI:A
--RPDFCLEPP-YTGPCKARIIRYAYNAKAGLCQTFVYGGCRA-KRNNFKSAEDCMRTCGGA
>PDB:1FAN:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLCQTFVYGGCRA-KRNNAKSAEDCMRTCGGA
>PDB:1NAG:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLCQTFVYGGCRA-KRGNFKSAEDCMRTCG--
>PDB:7PTI:A
--RPDFCLEPP-YTGPCKARIIRYFYNAKAGLAQTFVYGGCRA-KRNNFKSAEDAMRTCGGA
>PDB:8PTI:A
--RPDFCLEPPYT-GPCKARIIRYFYNAKAGLCQTFVGGGC-RAKRNNFKSAEDCMRTCGGA
      13


### Generating a Profile Hidden Markov Model Based on Structural Alignment
Now that we have a 'cleaned' seed file we will run the HMM build. This generates our HMM based on strucutral alignment

```hmmbuild <outfile> <infile>```


In [7]:
#hmmbuild bpti-kunitz.hmm bpti-kunitz-aln-clean.fasta

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             bpti-kunitz-aln-clean.fasta
# output HMM file:                  bpti-kunitz.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     bpti-kunitz-aln-clean    13    62    58     1.72  0.961 

# CPU time: 0.09u 0.01s 00:00:00.10 Elapsed: 00:00:00.29


### Checking the Model

Now we can see that the model is 58 matches (=lenght of our aln) and it has all the info about the positions of the transitons

![](./imgs/5.png)

Extract the protein structure ID + chain from the first member of each cluster 

### Generating the Sequence Logo 

Go to [skylign.org](http://skylign.org/) and upload your  .hmmer file. And check the box 'information content all' 

![](./imgs/6.png)

You can choose either "A static image of the logo scaled to the theoretical maximum height." In my case that logo looks like this: 

![](./imgs/logo_scaled_to_max.png)

Or "A static image of the logo" 
![](./imgs/static.png)

By studying the seq logo - we can see that the 6 cysteins are there. The ones that are important for keeping the structure compact are there. Then there other sites that are potentially important for the function of the protein such as the tyrosines (Y). But the cysteins seem to be the most important ones.


May include more options... We kept the hmm command  very basic we may include **more options**

#### Questions

Lorenzo: "OK what about the performance.py script? My performance.py script is running and i botained as best values, both for train and test 10-11" Prof Capriotti deems the value acceptable.


## Retrieving the Positive Set

We have to create the **positive** and **negative** datasets. The positive dataset **includes all proteins** that **contain Kunitz** domains within their structures, while the **negative** dataset is composed of proteins that do **not include** it. The input of the advanced search is:
* Cross-references > Family and domain databases > Pfam
 (PF00014)
* Reviewed > Reviewed

We retrieved **359** entries that contain the kunitz domain. 



In [1]:
grep '>' all_PF00014.fasta | wc -l | cat

     359


In [6]:
head all_PF00014.fasta

>sp|Q95241|A4_SAISC Amyloid-beta A4 protein OS=Saimiri sciureus OX=9521 GN=APP PE=2 SV=1
MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTK
TCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRDRKQCKTHPHIVIPYRCLVG
EFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFR
GVEFVCCPLAEESDHVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEE
EADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPC
RAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSVIPTTAASTPDAVDKYL
ETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQAKNLPKADKKAVIQHFQEKV
ESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITALQAVPPRPRHVFNMLKKYVR
AEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYERMNQSLSLLYNVPAVAEEIQ


## Retrieving the Negative Set

The **negative set** was retrieved from UniProt with the following query:
* NOT Cross-references > Family and domain databases > Pfam
 (PF00014)
* Reviewed > Reviewed
* search term ```NOT database:(type:pfam pf00014) AND reviewed:yes```

The search returned **561894** entries.

In [11]:
head NOT_PF00014.fasta 

>sp|Q4R8P0|ABHGA_MACFA Phosphatidylserine lipase ABHD16A OS=Macaca fascicularis OX=9541 GN=ABHD16A PE=2 SV=1
MAKLLSCVLGPRLYKIYRERDSERAPASVPETPTAVTAPHSSSWDTYYQPRALEKHADSI
LALASVFWSISYYSSPFAFFYLYRKGYLSLSKVVPFSHYAGTLLLLLAGVACLRGIGRWT
NPQYRQFITILEATHRNQSSENKRQLANYNFDFRSWPVDFHWEEPSSRKESRGGPSRRGV
ALLRPEPLHRGTADTLLNRVKKLPCQITSYLVAHTLGRRMLYPGSVYLLQKALMPVLLQG
QARLVEECNGRRAKLLACDGNEIDTMFVDRRGTAQPQGQKLVICCEGNAGFYEVGCISTP
LEAGYSVLGWNHPGFAGSTGVPFPQNEANAMDVVVQFAIHRLGFQPQDIIIYAWSIGGFT
ATWAAMSYPDVSAVILDASFDDLVPLALKVMPDSWRGLVTRTVRQHLNLNNAEQLCRYLG
PVLLIRRTKDEIITTTVPEDIMSNRGNDLLLKLLQHRYPRVMAEEGLQVVRQWLEASSQL
EEASIYSRWEVEEDWCLSVLRSYQAEHGPDFPWSVGEDMSADGRRQLALFLARKHLHNFE


In [4]:
grep '>' NOT_PF00014.fasta | wc -l

  561894


### Changing the Format of Positive and Negative Set

This command is to change the header of the positive set we retrieved from uniprot:
```awk if the substring($0,1,1)==">" ```  ...If substring of  $0(='whole line of arguments) from el 1 to el 1 (first char) == '>' 

```
{split ($0,a,"|")     $0 (=whole line), defining **a** as my var and split on '|' **returns list** of elemets (that is containd in **var a**)
```
the colon indicates end of cmd

```
; <to start a new statement> print '>'a[2]} ``` ... # print > and element 2 of list  beware that indexing here starts at 1 (not 0) ... so if we are in the col that contains the id ('>') print ">" and a[2] else just print the seq...

In [2]:
awk '{if (substr($0,1,1)==">"){split($0,a,"|"); print ">"a[2]} else {print $0}}' all_PF00014.fasta | less > positive_set.fasta


In [7]:
head positive_set.fasta

>Q95241
MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTK
TCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRDRKQCKTHPHIVIPYRCLVG
EFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFR
GVEFVCCPLAEESDHVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEE
EADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPC
RAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSVIPTTAASTPDAVDKYL
ETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQAKNLPKADKKAVIQHFQEKV
ESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITALQAVPPRPRHVFNMLKKYVR
AEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYERMNQSLSLLYNVPAVAEEIQ


In [3]:
awk '{if (substr($0,1,1)==">"){split($0,a,"|"); print ">"a[2]} else {print $0}}' NOT_PF00014.fasta | less > negative_set.fasta


In [15]:
head -20 negative_set.fasta
echo ''
echo 'number of sequences in negative_set.fasta'
grep ">" negative_set.fasta | wc -l | cat

>Q4R8P0
MAKLLSCVLGPRLYKIYRERDSERAPASVPETPTAVTAPHSSSWDTYYQPRALEKHADSI
LALASVFWSISYYSSPFAFFYLYRKGYLSLSKVVPFSHYAGTLLLLLAGVACLRGIGRWT
NPQYRQFITILEATHRNQSSENKRQLANYNFDFRSWPVDFHWEEPSSRKESRGGPSRRGV
ALLRPEPLHRGTADTLLNRVKKLPCQITSYLVAHTLGRRMLYPGSVYLLQKALMPVLLQG
QARLVEECNGRRAKLLACDGNEIDTMFVDRRGTAQPQGQKLVICCEGNAGFYEVGCISTP
LEAGYSVLGWNHPGFAGSTGVPFPQNEANAMDVVVQFAIHRLGFQPQDIIIYAWSIGGFT
ATWAAMSYPDVSAVILDASFDDLVPLALKVMPDSWRGLVTRTVRQHLNLNNAEQLCRYLG
PVLLIRRTKDEIITTTVPEDIMSNRGNDLLLKLLQHRYPRVMAEEGLQVVRQWLEASSQL
EEASIYSRWEVEEDWCLSVLRSYQAEHGPDFPWSVGEDMSADGRRQLALFLARKHLHNFE
ATHCTPLPAQNFQMPWHL
>P03949
MGHSHSTGKEINDNELFTCEDPVFDQPVASPKSEISSKLAEEIERSKSPLILEVSPRTPD
SVQMFRPTFDTFRPPNSDSSTFRGSQSREDLVACSSMNSVNNVHDMNTVSSSSSSSAPLF
VALYDFHGVGEEQLSLRKGDQVRILGYNKNNEWCEARLYSTRKNDASNQRRLGEIGWVPS
NFIAPYNSLDKYTWYHGKISRSDSEAILGSGITGSFLVRESETSIGQYTISVRHDGRVFH
YRINVDNTEKMFITQEVKFRTLGELVHHHSVHADGLICLLMYPASKKDKGRGLFSLSPNA
PDEWELDRSEIIMHNKLGGGQYGDVYEGYWKRHDCTIAVKALKEDAMPLHEFLAEAAIMK
DLHHKNLVRLLGVCTHEAPFYIITEFMCNGNLLEYLRRTDKSLLPPIILV

### Creating DB and BLAST seach to Find Proteins Already Used in HMM Building

(21.04.2020)

* first we generate a database (format db) the positive set.
* then we run BLAST on the 359 seqs (positive_set.fasta) with of the pdb and remove all sequences that have and E-value that is higher than a given threshold.

We have to **remove** the sequences that were used for the **MSA for the positive**. Keeping them in the testing set would bias the results!!! So we transform the positive set into a database in order to blast the sequences from the MSA to them.

In [19]:
grep ">" positive_set.fasta | wc -l

     359


In [ ]:
# all 359 BPTI sequences 
formatdb -i positive_set.fasta

In [5]:
# kunitz_pdb_hmm_input.fasta contains 13 structures
blastpgp -i kunitz_pdb_hmm_input.fasta -d positive_set.fasta -m 8 -e 0.001 -o positive_set.bl8

Now doing the same again but with all BPTI structures from the PDB to see if there is a difference between the outcome from above

In [3]:
# only_bpti_pdb.fasta contains 21 structures
blastpgp -i only_bpti_pdb.fasta -d positive_set.fasta -m 8 -e 0.001 -o positive_set_2.bl8

In [4]:
# tail -10 kunitz_pdb_hmm_input.fasta
grep ":" kunitz_pdb_hmm_input.fasta | wc -l 

      13


In [12]:
tail -20 positive_set.bl8

PDB:8PTI:A	Q868Z9	30.19	53	37	0	3	55	1788	1840	3e-05	36.2
PDB:8PTI:A	P00982	41.82	55	32	0	3	57	3	57	4e-08	45.4
PDB:8PTI:A	P36992	39.22	51	31	0	5	55	236	286	4e-08	45.4
PDB:8PTI:A	P82968	37.74	53	33	0	3	55	142	194	4e-08	45.1
PDB:8PTI:A	P0C5J5	37.74	53	33	0	3	55	361	413	5e-08	45.1
PDB:8PTI:A	D2Y488	41.07	56	33	0	1	56	21	76	5e-08	45.1
PDB:8PTI:A	B5KL29	35.09	57	37	0	1	57	27	83	5e-08	45.1
PDB:8PTI:A	P19859	36.36	55	35	0	1	55	1	55	5e-08	45.1
PDB:8PTI:A	P0DL86	36.54	52	33	0	4	55	3	54	5e-08	45.1
PDB:8PTI:A	B5L5Q1	38.60	57	35	0	1	57	27	83	6e-08	44.7
PDB:8PTI:A	D2Y490	41.07	56	33	0	1	56	3	58	6e-08	44.7
PDB:8PTI:A	B5KL27	35.09	57	37	0	1	57	27	83	7e-08	44.3
PDB:8PTI:A	P12111	31.48	54	37	0	3	56	3110	3163	8e-08	44.3
PDB:8PTI:A	P81162	33.33	54	36	0	5	58	8	61	1e-07	43.9
PDB:8PTI:A	P81906	44.68	47	26	0	5	51	5	51	1e-07	43.5
PDB:8PTI:A	G9I929	36.21	58	36	1	1	57	27	84	1e-07	43.5
PDB:8PTI:A	Q96NZ8	39.62	53	32	0	3	55	357	409	1e-07	43.5
PDB:8PTI:A	Q96NZ8	30.43	46	32	0	11	56	307	352	8e-05	34.3
PDB:8PTI:A	P0DJ

### Analyzing the BLAST Outfile


Analyse the BLAST search 

and eliminate the proteins that have a sequence ID above a decided threashold. The threashold is chosen according to how rigid we want to be in eliminating similar sequences. As we want to keep the dataset as large as possible we decided to eliminate just the sequences that are 99% identical to the ones used for the HMM. We have retrieved 11 sequences with 10% sequence identity.

In [19]:
# so these are the ones that are redundant
awk '{if ($3>=100) print $1}' positive_set.bl8 | sort -u | wc


       4       4      44


In this way we extract the protein ID to eliminate and we add them in a file.

In [11]:
awk '{if ($3>=100) print $2}' positive_set.bl8 | sort -u > IDs_of_eliminated.txt

In [12]:
awk '{if ($3>=100) print $2}' positive_set_2.bl8 | sort -u > IDs_of_eliminated2.txt

In [13]:
head IDs_of_eliminated.txt

P00974
P12111
P31713
P81658


In [15]:
awk '{if ($3>=100) print $2}' positive_set.bl8 | sort -u > IDs_of_eliminated2.txt

In [17]:
head IDs_of_eliminated2.txt

P00974
P12111
P31713
P81658


So it doesnt make a difference weather you use the file of the best representative of the cluster of pdb structures ```kunitz_pdb_hmm_input.fasta``` or all pdb structures (except APP) ``` only_bpti_pdb.fasta ```


Then we use this script to eliminate the proteins that have a sequence  identity => 100%. The program takes as input the file and a list of protein IDs that will be ignored in the copy of the file.

In [ ]:
#!/usr/bin/python
import sys


def get_ids(idfile):
	ids = open(idfile).read().rstrip().split()
	return ids

def print_seq(ids, dbfile):
	count = 0
	with open(dbfile,'r') as fdb:	#with is better then just open
		for line in fdb:
			if line[0] == '>':
				#pid=line.split('|')[1]
				pid = line[1:].rstrip()
			if pid not in ids:
				print (line.rstrip())


if __name__ == '__main__':
	idfile=sys.argv[1]
	dbfile = sys.argv[2]
	ids = get_ids(idfile)
	print_seq(ids, dbfile)


In [3]:
grep '>' positive_set.fasta |wc -l

     359


In [7]:
#python extractSeqProject.py IDs_of_eliminated.txt positive_set.fasta > positives_without.fasta


Now the 4 sequences were **removed**: 

In [8]:
grep '>' positives_without.fasta | wc -l |cat # he called it clean_positives or something similar.

     355


# Splitting the Positive Dataset
Now we split our positive dataset into two parts after having randomised the protein IDs. Then we check that there are not overlaps with the last command.

In [18]:
grep ">" positives_without.fasta | sed 's/>//' | sort -R > random_positives.txt
head -n 177 random_positives.txt > r1_positive.txt
tail -n +178 random_positives.txt > r2_positive.txt
#comm <(sort set_r1_positive.txt) <(sort set_r2_positive.txt)| awk -F '\t' '{print $3}' |wc

In [19]:
wc -l r1_positive.txt

     177 r1_positive.txt


In [20]:
wc -l r2_positive.txt

     178 r2_positive.txt


## Creating .fasta of r1_positive and r2_positive

The dataset is not too large, so we can use the same (slow) script above to extract the sequences from the **positive** set using the list of IDs. Pay attenion **line 16** must be **modified** to: 

> if pid in ids:

So only the protein in the list is selected but the other half is left.

In [4]:
python extractSeq2.py r1_positive.txt positives_without.fasta > r1_positive.fasta
python extractSeq2.py r2_positive.txt positives_without.fasta > r2_positive.fasta

In [7]:
head r1_positive.fasta
grep '>' r1_positive.fasta | wc -l


>Q95241
MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTK
TCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRDRKQCKTHPHIVIPYRCLVG
EFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFR
GVEFVCCPLAEESDHVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEE
EADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPC
RAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSVIPTTAASTPDAVDKYL
ETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQAKNLPKADKKAVIQHFQEKV
ESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITALQAVPPRPRHVFNMLKKYVR
AEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYERMNQSLSLLYNVPAVAEEIQ
     178


In [8]:
head r2_positive.fasta
grep '>' r1_positive.fasta | wc -l

>Q5IS80
MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTK
TCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRGRKQCKTHPHFVIPYRCLVG
EFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFR
GVEFVCCPLAEESDNVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEE
EADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPC
RAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSVMSQSLLKTTQEPLARD
PVKLPTTAASTPDAVDKYLETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQA
KNLPKADKKAVIQHFQEKVESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITAL
QAVPPRPRHVFNMLKKYVRAEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYER
     178


# Splitting the Negative Set

We generate a file containing **only the protein ID** of all 561894 **negative** Swissprot entries: using a pipeline that takes 
,  is used to 
The negative dataset needs less analysis. From the FASTA file containing all the 561894 entries the headers were reduced to the protein ID using the first command below. Then we extracted the protein IDs and we sorted them randomly. We splited the list of IDs in two half. 

In [1]:
head negative_set.fasta
grep ">" negative_set.fasta | wc -l

>Q4R8P0
MAKLLSCVLGPRLYKIYRERDSERAPASVPETPTAVTAPHSSSWDTYYQPRALEKHADSI
LALASVFWSISYYSSPFAFFYLYRKGYLSLSKVVPFSHYAGTLLLLLAGVACLRGIGRWT
NPQYRQFITILEATHRNQSSENKRQLANYNFDFRSWPVDFHWEEPSSRKESRGGPSRRGV
ALLRPEPLHRGTADTLLNRVKKLPCQITSYLVAHTLGRRMLYPGSVYLLQKALMPVLLQG
QARLVEECNGRRAKLLACDGNEIDTMFVDRRGTAQPQGQKLVICCEGNAGFYEVGCISTP
LEAGYSVLGWNHPGFAGSTGVPFPQNEANAMDVVVQFAIHRLGFQPQDIIIYAWSIGGFT
ATWAAMSYPDVSAVILDASFDDLVPLALKVMPDSWRGLVTRTVRQHLNLNNAEQLCRYLG
PVLLIRRTKDEIITTTVPEDIMSNRGNDLLLKLLQHRYPRVMAEEGLQVVRQWLEASSQL
EEASIYSRWEVEEDWCLSVLRSYQAEHGPDFPWSVGEDMSADGRRQLALFLARKHLHNFE
  561894


In [2]:
grep ">" negative_set.fasta | sed 's/>//' | sort -R > random.txt
head -n 280947 random.txt > r1_negative_set.txt 
tail -n +280948 random.txt > r2_negative_set.txt

In [9]:
#confirming it worked
wc -l r1_negative_set.txt
wc -l r2_negative_set.txt

  280947 r1_negative_set.txt
  280947 r2_negative_set.txt


Extracting sequences from the protein IDs of such a large file would take too much time with the script we used for the positive ones. A better idea is to download pyfasta. After having installed pyfasta we can use these commands to extract the sequences:

In [12]:
#pip install pyfasta
pyfasta extract --header --fasta negative_set.fasta --file r1_negative_set.txt > r1_negative.fasta
pyfasta extract --header --fasta negative_set.fasta --file r2_negative_set.txt > r2_negative.fasta

The pyfasta extract command works exactly like the python program above, but it is faster as it previously organises the files and then computes the search. The command takes the FASTA file as input and another file containing the list of identifiers and it returns another file with the extracted sequences. 


echo 'Head r1: '
head r1_negative.fasta
echo ''
echo 'Head r2: '
head r2_negative.fasta

# HMMSEARCH

It takes as input an **hmmfile** and a fasta sequence input.

To facilitate fileparsing and help the program to work in a more robust manner we suggests the following options:
- --tblout <f>     : saves parseable table of per-sequence hits to file <f>
- --noali          : don't output alignments, so output is smaller, because we only want the E-value for classification.
- --max    : Turn all heuristic filters off (less speed, more power)

In [2]:
hmmsearch -h | cat

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Usage: hmmsearch [options] <hmmfile> <seqdb>

Basic options:
  -h : show brief help on version and usage

Options directing output:
  -o <f>           : direct output to file <f>, not stdout
  -A <f>           : save multiple alignment of all hits to file <f>
  --tblout <f>     : save parseable table of per-sequence hits to file <f>
  --domtblout <f>  : save parseable table of per-domain hits to file <f>
  --pfamtblout <f> : save table of hits and domains to file, in Pfam format <f>
  --acc            : prefer accessions over names in output
  --noali          : don't output alignments, so output is smaller
  --notextw        : unlimit ASCII text output line width
  --textw <n>      : set max width of AS

In [ ]:
# first positive set Training
hmmsearch -Z 1 --noali --max --tblout r1_positive.hits bpti-kunitz.hmm r1_positive.fasta 

In [ ]:
# second positive Testing
hmmsearch -Z 1 --noali --max --tblout r2_positive.hits bpti-kunitz.hmm r2_positive.fasta

In [ ]:
# second negative Training
hmmsearch -Z 1 --noali --max --tblout r1_negative.hits bpti-kunitz.hmm r1_negative.fasta

In [ ]:
# second negative  Testing
hmmsearch -Z 1 --noali --max --tblout r2_negative.hits bpti-kunitz.hmm r2_negative.fasta

## Generating one file that contains only the evalue, the name of the sequence and the type of the sequence: 

This serves to find the optimal threshold to be able to distinguish between negatives (0) and positives (1) as classified by PFAM/Uniprot. So we extract:
- the protein ID \$1,  
- E-value (of the single domain) from  \$8

And we add the lable 1 (pos) or 0 (neg) to the corresponding entries. 

#### positives:

In [46]:
grep -v "^#" r1_positive.hits | awk '{print $1,$8,1}'> r1_positive.out
grep -v "^#" r2_positive.hits | awk '{print $1,$8,1}'> r2_positive.out
echo "The positives r1:"
head -2 r1_positive.out
wc -l r1_positive.out

echo ''
echo "The positives r2:"
head -2 r2_positive.out
wc -l r2_positive.out


The positives r1:
Q868Z9 4.6e-21 1
O76840 3e-22 1
     178 r1_positive.out

The positives r2:
O54819 3.1e-25 1
P10646 1.6e-24 1
     177 r2_positive.out


#### negatives:

In [47]:
# grep -v "^#" r1_negative.hits | awk '{print $1,$8,0}'> r1_negative.out
# grep -v "^#" r2_negative.hits | awk '{print $1,$8,0}'> r2_negative.out
echo 'The negatives r1:'
head -2 r1_negative.out
wc -l r1_negative.out

echo ''
echo 'The negatives r2:'
head -2 r2_negative.out
wc -l r2_negative.out

The negatives r1:
P0DJ63 3.9e-08 0
P84555 9.2e-08 0
  135429 r1_negative.out

The negatives r2:
G3LH89 4.2e-23 0
P83605 4.6e-08 0
  135945 r2_negative.out


### Recovering IDs with an E-value above 10

Each file contains however only elements that had an e-value <= 10 
Now we need to generate a file with the same layout [<id> <e-value of the single domain> <0 or 1>] containing the IDs of the proteins that were **above** the cut-off E-value of 10 and thus not reported in the ```rX_negative.out```.
    
This **obviously** does not need to be done with the positive set as none of the evalues returned are higher than 10 ;-)

In [3]:
#comm <(grep "^>" r2_negative.fasta | sed 's/>//' | sort) <(awk '{print $1}' r2_negative.out | sort) | awk -F '\t' '{if ($1!="") print $1,10,0}' > missing_negative_set2.out
#comm <(grep "^>" r1_negative.fasta | sed 's/>//' | sort) <(awk '{print $1}' r1_negative.out | sort) | awk -F '\t' '{if ($1!="") print $1,10,0}' > missing_negative_set1.out
cat missing_negative_set1.out | wc -l
cat missing_negative_set2.out | wc -l

  145518
  145002


### Checking weather everything sums up and makes sense: remember the total amount of negative sequences downloaded was 561894!
summing the ID's that were below the threashold with the IDs above the threashold:

In [34]:
echo "Negative Set 1"
head -3 total_negative_set2.out
wc -l missing_negative_set1.out
wc -l r1_negative.out
echo ''
echo "Negative Set 2"
wc -l missing_negative_set2.out
wc -l r2_negative.out


Negative Set 1
head: total_negative_set2.out: No such file or directory
  145518 missing_negative_set1.out
  135429 r1_negative.out

Negative Set 2
  145002 missing_negative_set2.out
  135945 r2_negative.out


In [37]:
echo 'Sum negative set 1:'
expr 145518 + 135429 

echo ''
echo 'Sum negative set 2:'
expr 145002 + 135945

echo ''
echo 'Total negatives'
expr 280947 + 280947

Sum negative set 1:
280947

Sum negative set 2:
280947

Total negatives
561894


#### The sum checks out every thing should be alright

Now we need to join the two files using cat

In [40]:
cat missing_negative_set1.out r1_negative.out > both_r1.out
cat missing_negative_set2.out r2_negative.out > both_r2.out
wc -l both_r1.out
head -3 both_r1.out
wc -l both_r2.out
head -3 both_r2.out

  280947 both_r1.out
A0A009IHW8 10 0
A0A023IWG2 10 0
A0A023IWI8 10 0
  280947 both_r2.out
A0A023IWE1 10 0
A0A023IWM4 10 0
A0A023PXD3 10 0


The py script below will be used to test the performance.

In [ ]:
#!/usr/bin/python
import sys
import math

def get_hmm(filename):
    ''' Input: file containing 3 columns: $1 contains UniprotID, 
    $2 E-vlaue (domain evalue) of the hmmsearch and $3 '0' negatives OR '1' positives.
    It returns the f_list containing the best evalue and the kind of the the sequence (0 or 1) '''
    f_list=[]
    d={}
    with open(filename) as f:
        for line in f:
            v=line.rstrip().split()
            d[v[0]]=d.get(v[0],[])
            d[v[0]].append([ float(v[1]),int(v[2]), v[0] ]) # eval, kind, ID
            # print(d[v[0]])
        for ids in d.keys():
        # we might get many hits for one single uniprot ID but we 
        # only keep the one with the lowest e-value
            d[ids].sort()
            f_list.append(d[ids][0])
        return f_list  # = data

def get_conf_mtrx(data,threashold):
    '''Calculates the confusion matrix from a list of lists (data) containing one list for each seuqence: containing: e-value,  kind: either 1 = positive or 0 = negative and ID. This file is used to calculate the confusion matrix. threashold = the selected threshold to calculate the values that are above and below the threshold [[TP, FP], [FN, TN]]'''
    cm = [[0.0,0.0],[0.0,0.0]] #building zero matrix first
    n = 0
    m = 0
    eval = 0
    kind = 1
    for i in data: # for every list in 'data'
        if i[eval]<th and i[kind]==1: # lower than threshold and true positive, 1
            cm[0][0] += 1
        if i[eval]>=th and i[kind]==1: # false negative
            cm[1][0] += 1
            n +=1
            if n < 10:
                print(i[2], 'FN') # to save the ID  of the FN
        if i[0]<th and i[1]==0: # true negative
            cm[0][1] += 1
            m += 1
            if m < 10:
                print(i[2], 'FP') # to save ID of the FP
        if i[0]>=th and i[1] ==0:
            cm[1][1] += 1
    return cm

def accuracy(m): 
    '''Takes the confusion matrix as input and calculates the accuracy
    (TP + TN) / (TP + FP + FN + TN) '''
    return float(m[0][0]+m[1][1])/(sum(m[0])+sum(m[1]))

def matthew_cc(m):
    '''Takes the confusion matrix as input and returns the Matthews correlation coefficient'''
    d=(m[0][0]+m[1][0])*(m[0][0]+m[0][1])*(m[1][1]+m[1][0])*(m[1][1]+m[0][1])
    return float((m[0][0]*m[1][1]-m[0][1]*m[1][0])/math.sqrt(d))

def tpr(cm):
    return cm[0][0]/(cm[0][0]+cm[1][0])

def fpr(cm):
    return cm[0][1]/(cm[0][1]+cm[1][1])

def ppv(cm):
    '''returns the positive predictive value = tp/ (tp + fp) = n of tp / all positive calls'''
    return cm[0][0]/(cm[0][0]+cm[0][1])

def npv(cm):
    '''returns the positive predictive value = tn/ (tn + fn) = n of tn / all negative calls'''
    return cm[1][1]/(cm[1][1]+cm[1][0])

if __name__== "__main__":
    filename=sys.argv[1]
    #th=float(sys.argv[2])
    data = get_hmm(filename)
    for i in range(20): # provides 20 diff e-vla threasholds
        th=10**-i  # initial threashold: th
        cm= get_conf_mtrx(data,th) # takes eval and class (neg or pos)
        print('Threshold:',th,'\nACC:', accuracy(cm),'\nMatthews:',matthew_cc(cm), "\nTPR:", tpr(cm), '\nFPR:', fpr(cm), '\nPositive pred val:', ppv(cm), "\nThe Matrix:", cm,"\n")

In [4]:
python performance.py <(cat both_r1.out r1_positive.out)

P0DJ63 FP
P84555 FP
P85040 FP
P36235 FP
P71089 FP
Q9PS05 FP
P40500 FP
Q91845 FP
Q91571 FP
Threshold: 1 
ACC: 0.7162472209871054 
Matthews: 0.03992851546858927 
TPR: 1.0 
FPR: 0.28393255667439055 
Positive pred val: 0.002226447190673938 
The Matrix: [[178.0, 79770.0], [0.0, 201177.0]] 

P0DJ63 FP
P84555 FP
P85040 FP
P36235 FP
P71089 FP
Q9PS05 FP
P40500 FP
Q91845 FP
Q91571 FP
Threshold: 0.1 
ACC: 0.9621378390395732 
Matthews: 0.12579676409302057 
TPR: 1.0 
FPR: 0.03788614934489423 
Positive pred val: 0.01644797634448346 
The Matrix: [[178.0, 10644.0], [0.0, 270303.0]] 

P0DJ63 FP
P84555 FP
P85040 FP
P36235 FP
P71089 FP
Q9PS05 FP
P40500 FP
Q91845 FP
Q91571 FP
Threshold: 0.01 
ACC: 0.9958381502890173 
Matthews: 0.36262593371279445 
TPR: 1.0 
FPR: 0.004164486540165939 
Positive pred val: 0.13204747774480713 
The Matrix: [[178.0, 1170.0], [0.0, 279777.0]] 

P0DJ63 FP
P84555 FP
P85040 FP
P36235 FP
P71089 FP
Q9PS05 FP
P40500 FP
Q91845 FP
Q91571 FP
Threshold: 0.001 
ACC: 0.9996905291240551 
Mat

In [5]:
cat performance.py

#!/usr/bin/python
import sys
import math

def get_hmm(filename):
    ''' Input: file containing 3 columns: $1 contains UniprotID, 
    $2 E-vlaue (domain evalue) of the hmmsearch and $3 '0' negatives OR '1' positives.
    It returns the f_list containing the best evalue and the kind of the the sequence (0 or 1) '''
    f_list=[]
    d={}
    with open(filename) as f:
        for line in f:
            v=line.rstrip().split()
            d[v[0]]=d.get(v[0],[])
            d[v[0]].append([ float(v[1]),int(v[2]), v[0] ]) # eval, kind, ID
            # print(d[v[0]])
        for ids in d.keys():
        # we might get many hits for one single uniprot ID but we 
        # only keep the one with the lowest e-value
            d[ids].sort()
            f_list.append(d[ids][0])
        return f_list  # = data

def get_conf_mtrx(data,threashold):
    '''Calculates the confusion matrix from a list of lists (data) containing one list for each seuqence: containing: e-value,  kind: either 1 = posit

In [6]:
 python performance.py <(cat both_r2.out r2_positive.out)

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 1 
ACC: 0.7160541255815939 
Matthews: 0.0397976652837338 
TPR: 1.0 
FPR: 0.28412476374547513 
Positive pred val: 0.0022124723440956986 
The Matrix: [[177.0, 79824.0], [0.0, 201123.0]] 

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 0.1 
ACC: 0.9618246752322819 
Matthews: 0.12492135595680638 
TPR: 1.0 
FPR: 0.038199375682957996 
Positive pred val: 0.01622513520946008 
The Matrix: [[177.0, 10732.0], [0.0, 270215.0]] 

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 0.01 
ACC: 0.9959661928543988 
Matthews: 0.3666967253993352 
TPR: 1.0 
FPR: 0.004036348492776218 
Positive pred val: 0.13501144164759726 
The Matrix: [[177.0, 1134.0], [0.0, 279813.0]] 

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 0.001 
ACC: 0.9997545567080719 
Mat

In [7]:
python performance.py <(cat both_r2.out r2_positive.out)

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 1 
ACC: 0.7160541255815939 
Matthews: 0.0397976652837338 
TPR: 1.0 
FPR: 0.28412476374547513 
Positive pred val: 0.0022124723440956986 
The Matrix: [[177.0, 79824.0], [0.0, 201123.0]] 

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 0.1 
ACC: 0.9618246752322819 
Matthews: 0.12492135595680638 
TPR: 1.0 
FPR: 0.038199375682957996 
Positive pred val: 0.01622513520946008 
The Matrix: [[177.0, 10732.0], [0.0, 270215.0]] 

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 0.01 
ACC: 0.9959661928543988 
Matthews: 0.3666967253993352 
TPR: 1.0 
FPR: 0.004036348492776218 
Positive pred val: 0.13501144164759726 
The Matrix: [[177.0, 1134.0], [0.0, 279813.0]] 

G3LH89 FP
P83605 FP
P85039 FP
P56409 FP
P84556 FP
P0DM47 FP
Q9FF80 FP
Q91694 FP
Q91736 FP
Threshold: 0.001 
ACC: 0.9997545567080719 
Mat